In [2]:
import os
import glob
from functools import lru_cache
from sqlalchemy import create_engine, text
from typing import List

# ── CONFIGURE ────────────────────────────────────────────────────────────────
# Database configuration - adjust paths as needed
def get_db_path():
    """Get database path, handling both script and notebook contexts."""
    try:
        # When running as a script
        script_dir = os.path.dirname(__file__)
        return os.path.join(script_dir, '..', 'data', 'schedules.db')
    except NameError:
        # When running in notebook or interactive environment
        # Update this path to match your actual database location
        return os.path.abspath('data/schedules.db')  # Adjust this path as needed

DB_PATH = get_db_path()
DB_URL = f"sqlite:///{DB_PATH}"
SEMESTER = "sp25"  # Update as needed

print(f"Database path: {DB_PATH}")
print(f"Database URL: {DB_URL}")


Database path: /home/asj53/BOScheduling/myproject/app/data/schedules.db
Database URL: sqlite:////home/asj53/BOScheduling/myproject/app/data/schedules.db


In [3]:
import numpy as np
import pandas as pd
import math
import os
from matplotlib import pyplot as plt
import seaborn as sns
from itertools import product
from config.settings import SAVE_PATH, DATA_PATH, UI_PATH , SEMESTER
from globals.build_global_sets import normalize_and_merge
times = ['']
def slots_to_time(slots):
    d = {}
    if 'fa' in SEMESTER : 
        d=  {
            1: 'Dec 13, 9am', 
            2: 'Dec 13, 2pm',
            3: 'Dec 13, 7pm',
            4: 'Dec 14, 9am',
            5: 'Dec 14, 2pm',
            6: 'Dec 14, 7pm',
            7: 'Dec 15, 9am',
            8: 'Dec 15, 2pm',
            9: 'Dec 15, 7pm',
            10: 'Dec 16, 9am',
            11: 'Dec 16, 2pm',
            12: 'Dec 16, 7pm',
            13: 'Dec 17, 9am',
            14: 'Dec 17, 2pm',
            15: 'Dec 17, 7pm',
            16: 'Dec 18, 9am',
            17: 'Dec 18, 2pm',
            18: 'Dec 18, 7pm',
            19: 'Dec 19, 9am',
            20: 'Dec 19, 2pm',
            21: 'Dec 19, 7pm',
            22: 'Dec 20, 9am',
            23: 'Dec 20, 2pm',
            24: 'Dec 20, 7pm',
            25: 'Dec 21, 9am',
            26: 'Dec 21, 2pm',
            27: 'Dec 21, 7pm'}
    else:  
        d=   {
        1:  'May 11, 9am',
        2:  'May 11, 2pm',
        3:  'May 11, 7pm',
        4:  'May 12, 9am',
        5:  'May 12, 2pm',
        6:  'May 12, 7pm',
        7:  'May 13, 9am',
        8:  'May 13, 2pm',
        9:  'May 13, 7pm',
        10: 'May 14, 9am',
        11: 'May 14, 2pm',
        12: 'May 14, 7mm',
        13: 'May 15, 9am',
        14: 'May 15, 2pm',
        15: 'May 15, 7pm',
        16: 'May 16, 9am',
        17: 'May 16, 2pm',
        18: 'May 16, 7pm',
        19: 'May 17, 9am',
        20: 'May 17, 2pm',
        21: 'May 17, 7pm',
        22: 'May 18, 9am',
        23: 'May 18, 2pm',
        24: 'May 18, 7pm', 
        25: 'May 19, 9am', 
        26: 'May 19, 2pm',
        27: 'May 19, 7pm' }
    
    return [d[s] for s in slots]
# Create the chart
def get_plot(schedule_name, name):
  sched = pd.read_csv(SAVE_PATH +'/schedules/' + schedule_name )
  exam_sizes = pd.read_csv(DATA_PATH + '/exam_sizes.csv')
  slots = np.unique(sched['slot'].values)

  num_slots1 = len(slots)
  num_slots2 = int(max(slots))
  h = np.zeros(num_slots2)
  h1 = np.zeros(num_slots2)
  h2 = np.zeros(num_slots2)
  h3 = np.zeros(num_slots2)
  h4 = np.zeros(num_slots2)
  for s in slots:
      s = int(s)
      exams = sched[sched['slot']==s]['exam'].tolist()
      exams_over_400 = sched[(sched['slot']==s) & (sched['size']>= 400)]['exam'].tolist()
      exams_in_300_400 = sched[(sched['slot']==s) & (sched['size']>= 300) & (sched['size']< 400)]['exam'].tolist()
      exams_in_200_300 = sched[(sched['slot']==s) & (sched['size']>= 200) & (sched['size']< 300)]['exam'].tolist()
      exams_in_100_200 = sched[(sched['slot']==s) & (sched['size']>= 100) & (sched['size']< 200)]['exam'].tolist()
      sizes_over_400 = exam_sizes[exam_sizes['exam'].isin(exams_over_400)]['size'].sum()
      sizes_in_300_400 = exam_sizes[exam_sizes['exam'].isin(exams_in_300_400)]['size'].sum()
      sizes_in_200_300 = exam_sizes[exam_sizes['exam'].isin(exams_in_200_300 )]['size'].sum()
      sizes_in_100_200 = exam_sizes[exam_sizes['exam'].isin(exams_in_100_200 )]['size'].sum()
      sizes = exam_sizes[exam_sizes['exam'].isin(exams)]['size'].sum()
      h[s-1] = sizes
      h1[s-1] = sizes_over_400
      h2[s-1] = sizes_in_300_400
      h3[s-1] = sizes_in_200_300
      h4[s-1] = sizes_in_100_200

  plt.style.use('classic')
  plt.figure(figsize=(18, 12))

  # plt.bar(x=slots, height=[max(h)]*num_slots1, color='red', alpha=0.4, width = 1, align = 'center')       
  plt.bar(x=range(1,num_slots2+1), height=h1, align='center', width=1, 
          color = 'tab:red', label = "Exams w/ over 400 students")
  plt.bar(x=range(1,num_slots2+1), height=h2, align='center', width=1, 
          bottom = h1, color = 'tab:orange', label = "Exams w/ over 300 but less than 400 students")
  plt.bar(x=range(1,num_slots2+1), height=h3, align='center', width=1, 
          bottom = h1+h2, color = 'gold', label = "Exams w/ over 200 but less than 300 students")
  plt.bar(x=range(1,num_slots2+1), height=h4, align='center', width=1, 
          bottom = h1+h2+h3, color = 'pink', label = "Exams w/ over 100 but less than 200 students")

  plt.bar(x=range(1,num_slots2+1), height=h-h1-h2-h3-h4, align='center',
          bottom = h1+h2+h3+h4, width=1, color = 'tab:purple', label = "Other Exams")

  plt.xlabel('Times', fontsize=20)
  plt.xticks(np.arange(1, num_slots2 + 1), slots_to_time(np.arange(1, num_slots2 + 1)), rotation = 90, fontsize=16)
  plt.yticks(fontsize = 16)
  plt.ylabel('Number of students',  fontsize=20)
  plt.title('Number of students taking an exam in each time slot',  fontsize=25)
  plt.legend(loc = 'best', fontsize=14)
  plt.savefig(UI_PATH + name + '.png')
  
  plt.show()

def last_day(sched_name, save_name):
    #goop['Exam Block'] = 
    #sched, by_student_block = normalize_and_merge(goop,)
    sched = pd.read_csv(SAVE_PATH + '/schedules/' + sched_name)
    print(sched)
    enrl_df = pd.read_csv(DATA_PATH + '/enrl.csv')
    enrl_df = enrl_df.merge(sched, left_on = 'Exam Key', right_on = 'Exam Group')
    by_student_block = enrl_df.groupby('anon-netid')['slot'].apply(list).reset_index(name='slots') #create_by_student_slot_df(exam_df, schedule_dict)
    by_student_block['last_block'] = by_student_block['slots'].apply(lambda x: max(x)).copy()
    last_block_counts = by_student_block['last_block'].value_counts().reset_index()
    last_block_counts.columns = ['last_block', 'occurrences']

    last_block_counts = last_block_counts.sort_values(by='last_block').reset_index(drop=True)
    print('last_block_counts' , last_block_counts )

    slots = np.unique(sched['slot'].values)
    # Ensure num_slots2 is an integer for range function
    num_slots2 = int(max(slots)) if len(slots) > 0 else 0

    print('slot , ' , slots)
    h = np.zeros(num_slots2)

    # Convert last_block_counts to a dictionary for efficient lookup
    counts_dict = last_block_counts.set_index('last_block')['occurrences'].to_dict()

    for s in range(1, num_slots2 + 1): # Iterate through all possible slot numbers
        # Get the occurrence count from the dictionary, defaulting to 0 if not found
        h[s-1] = counts_dict.get(float(s), 0)

    plt.style.use('classic')
    plt.figure(figsize=(18, 12))
    plt.bar(x=range(1,num_slots2+1), height=h, align='center', width=1, color = 'pink')

    plt.xlabel('Times', fontsize=20)
    # Ensure the ticks cover all possible slots up to num_slots2
    plt.xticks(np.arange(1, num_slots2 + 1), slots_to_time(np.arange(1, num_slots2 + 1)), rotation = 90, fontsize=16)
    plt.yticks(fontsize = 16)
    plt.ylabel('Number of students',  fontsize=20)
    plt.title('Number of students taking their last exam in each time slot',  fontsize=25)
    plt.savefig(UI_PATH +save_name+ '_dist.png' )
    plt.show()
    

run_name:  thomp:
ASSIGNMENT_TYPE:  block
semester ,  sp25
co     Unnamed: 0  72-0103  1-9185  MLG17  1-8222  1-6240  1-5549  1-8021  \
0      72-0103        0       1     52       0       1       1       0   
1       1-9185        1       0      0       0       0       0       0   
2        MLG17       52       0      0       0      15       0       0   
3       1-8222        0       0      0       0       1       0       0   
4       1-6240        1       0     15       1       0       2       0   
..         ...      ...     ...    ...     ...     ...     ...     ...   
561     1-9455        0       0      0       0       0       0       0   
562     1-0003        0       0      0       0       0       0       0   
563    1-21192        0       0      0       0       0       0       0   
564     1-9034        0       0      0       0       0       0       0   
565    1-19927        0       0      0       0       0       0       0   

     1-8767  1-8181  ...  1-10951  1-9806  1-0055

In [17]:

# ───────────────────────────────────────────────────────────────────────────────

engine = create_engine(DB_URL, echo=False)  # Set echo=True for debugging

def get_schedule_files(date_prefix: str, semester: str = SEMESTER) -> List[str]:
    """Get schedule IDs from database that match the given date prefix."""
    print('Getting schedules for prefix:', date_prefix)
    
    with engine.begin() as conn:
        result = conn.execute(text("""
            SELECT DISTINCT s.schedule_id 
            FROM schedule_plots s
            JOIN metrics m ON s.schedule_id = m.schedule_id
            WHERE s.schedule_id LIKE :prefix 
            AND m.semester = :semester
            ORDER BY s.schedule_id
        """), {
            "prefix": f"{date_prefix}%",
            "semester": semester
        })
        
        schedule_ids = [row.schedule_id for row in result]
        print(f"Found {len(schedule_ids)} schedules matching prefix '{date_prefix}'")
        return schedule_ids

def check_plot_exists(schedule_id: str, plot_type: str = 'sched_plot') -> bool:
    """Check if a plot exists in the database for the given schedule_id."""
    with engine.begin() as conn:
        result = conn.execute(text("""
            SELECT 1 FROM schedule_plots 
            WHERE schedule_id = :schedule_id 
            AND :plot_type IS NOT NULL
        """), {
            "schedule_id": schedule_id,
            "plot_type": plot_type
        })
        return result.fetchone() is not None

def update_plot_in_db(schedule_id: str, plot_type: str, plot_filename: str, semester: str = SEMESTER):
    """Update the plot information in the database."""
    with engine.begin() as conn:
        # Check if record exists
        existing = conn.execute(text("""
            SELECT schedule_id FROM schedule_plots 
            WHERE schedule_id = :schedule_id
        """), {"schedule_id": schedule_id}).fetchone()
        
        if existing:
            # Update existing record
            if plot_type == 'sched_plot':
                conn.execute(text("""
                    UPDATE schedule_plots 
                    SET sched_plot = :plot_filename
                    WHERE schedule_id = :schedule_id
                """), {
                    "plot_filename": plot_filename,
                    "schedule_id": schedule_id
                })
            elif plot_type == 'last_plot':
                conn.execute(text("""
                    UPDATE schedule_plots 
                    SET last_plot = 1
                    WHERE schedule_id = :schedule_id
                """), {"schedule_id": schedule_id})
        else:
            # Insert new record
            sched_plot = plot_filename if plot_type == 'sched_plot' else None
            last_plot = 1 if plot_type == 'last_plot' else 0
            
            conn.execute(text("""
                INSERT INTO schedule_plots (schedule_id, sched_plot, last_plot, semester)
                VALUES (:schedule_id, :sched_plot, :last_plot, :semester)
            """), {
                "schedule_id": schedule_id,
                "sched_plot": sched_plot,
                "last_plot": last_plot,
                "semester": semester
            })

def plot_exists_on_disk(schedule_id: str, plot_suffix: str = '', plots_dir: str = None) -> bool:
    """Check if plot file exists on disk."""
    if plots_dir is None:
        try:
            # When running as a script
            plots_dir = os.path.join(os.path.dirname(__file__), 'static', 'plots')
        except NameError:
            # When running in notebook - adjust this path as needed
            plots_dir = os.path.abspath('static/plots')  # Update this path
    
    plot_filename = f"{schedule_id}{plot_suffix}.png"
    plot_path = os.path.join(plots_dir, plot_filename)
    return os.path.exists(plot_path)

@lru_cache(maxsize=1)
def generate_plots_for_files(date_prefix: str, semester: str = SEMESTER):
    """Generate missing schedule and distribution plots for all files matching prefix."""
    schedule_ids = get_schedule_files(date_prefix, semester)
    print("PLOT SCHEDULE IDs:", schedule_ids)
    
    for schedule_id in schedule_ids:
        # Check and generate regular schedule plot
        if not plot_exists_on_disk(schedule_id):
            print(f"Generating schedule plot for {schedule_id}")
            get_plot(schedule_id, schedule_id)  # Assuming get_plot function exists
            # Update database after successful plot generation
            plot_filename = f"{schedule_id}.png"
            update_plot_in_db(schedule_id, 'sched_plot', plot_filename, semester)
        
        # Check and generate distribution plot
        if not plot_exists_on_disk(schedule_id, '_dist'):
            print(f"Generating distribution plot for {schedule_id}")
            last_day(schedule_id, schedule_id)  # Assuming last_day function exists
            # Update database after successful plot generation
            update_plot_in_db(schedule_id, 'last_plot', f"{schedule_id}_dist.png", semester)

def get_schedule_plot_info(schedule_id: str) -> dict:
    """Get plot information for a specific schedule from the database."""
    with engine.begin() as conn:
        result = conn.execute(text("""
            SELECT schedule_id, sched_plot, last_plot, semester
            FROM schedule_plots
            WHERE schedule_id = :schedule_id
        """), {"schedule_id": schedule_id})
        
        row = result.fetchone()
        if row:
            return {
                'schedule_id': row.schedule_id,
                'sched_plot': row.sched_plot,
                'has_last_plot': bool(row.last_plot),
                'semester': row.semester
            }
        return None

def get_all_plots_for_prefix(date_prefix: str, semester: str = SEMESTER) -> List[dict]:
    """Get all plot information for schedules matching the prefix."""
    with engine.begin() as conn:
        result = conn.execute(text("""
            SELECT sp.schedule_id, sp.sched_plot, sp.last_plot, sp.semester,
                   s.display_name, s.max_slot
            FROM schedule_plots sp
            JOIN schedules s ON sp.schedule_id = s.schedule_id
            WHERE sp.schedule_id LIKE :prefix
            AND sp.semester = :semester
            ORDER BY sp.schedule_id
        """), {
            "prefix": f"{date_prefix}%",
            "semester": semester
        })
        
        plots = []
        for row in result:
            plots.append({
                'schedule_id': row.schedule_id,
                'sched_plot': row.sched_plot,
                'has_last_plot': bool(row.last_plot),
                'semester': row.semester,
                'display_name': row.display_name,
                'max_slot': row.max_slot
            })
        
        return plots

def cleanup_missing_plots(semester: str = SEMESTER):
    """Remove database entries for plots that no longer exist on disk."""
    try:
        # When running as a script
        plots_dir = os.path.join(os.path.dirname(__file__), 'static', 'plots')
    except NameError:
        # When running in notebook - adjust this path as needed
        plots_dir = os.path.abspath('static/plots')  # Update this path
    
    with engine.begin() as conn:
        # Get all plot records
        result = conn.execute(text("""
            SELECT schedule_id, sched_plot, last_plot
            FROM schedule_plots
            WHERE semester = :semester
        """), {"semester": semester})
        
        for row in result:
            schedule_id = row.schedule_id
            needs_update = False
            updates = {}
            
            # Check if sched_plot file exists
            if row.sched_plot:
                plot_path = os.path.join(plots_dir, row.sched_plot)
                if not os.path.exists(plot_path):
                    updates['sched_plot'] = None
                    needs_update = True
            
            # Check if distribution plot file exists
            if row.last_plot:
                dist_plot_path = os.path.join(plots_dir, f"{schedule_id}_dist.png")
                if not os.path.exists(dist_plot_path):
                    updates['last_plot'] = 0
                    needs_update = True
            
            # Update database if needed
            if needs_update:
                if 'sched_plot' in updates and 'last_plot' in updates:
                    conn.execute(text("""
                        UPDATE schedule_plots 
                        SET sched_plot = :sched_plot, last_plot = :last_plot
                        WHERE schedule_id = :schedule_id
                    """), {
                        "sched_plot": updates['sched_plot'],
                        "last_plot": updates['last_plot'],
                        "schedule_id": schedule_id
                    })
                elif 'sched_plot' in updates:
                    conn.execute(text("""
                        UPDATE schedule_plots 
                        SET sched_plot = :sched_plot
                        WHERE schedule_id = :schedule_id
                    """), {
                        "sched_plot": updates['sched_plot'],
                        "schedule_id": schedule_id
                    })
                elif 'last_plot' in updates:
                    conn.execute(text("""
                        UPDATE schedule_plots 
                        SET last_plot = :last_plot
                        WHERE schedule_id = :schedule_id
                    """), {
                        "last_plot": updates['last_plot'],
                        "schedule_id": schedule_id
                    })
                
                print(f"Updated plot records for {schedule_id}: {updates}")


In [18]:
"""Test the helper functions."""
print("Testing helper functions...")

# Test getting schedule files
test_prefix = "20240620"
schedules = get_schedule_files(test_prefix)
print(f"Found schedules: {schedules}")

# Test getting plot info
if schedules:
  plot_info = get_schedule_plot_info(schedules[0])
  print(f"Plot info for {schedules[0]}: {plot_info}")

# Test getting all plots for prefix
all_plots = get_all_plots_for_prefix(test_prefix)
print(f"All plots for prefix: {len(all_plots)} found")

Testing helper functions...
Getting schedules for prefix: 20240620
Found 0 schedules matching prefix '20240620'
Found schedules: []
All plots for prefix: 0 found


In [ ]:
import os
import glob
from functools import lru_cache
from sqlalchemy import create_engine, text
from typing import List

# ── CONFIGURE ────────────────────────────────────────────────────────────────
# Database configuration - adjust paths as needed
def get_db_path():
    """Get database path, handling both script and notebook contexts."""
    try:
        # When running as a script
        script_dir = os.path.dirname(__file__)
        return os.path.join(script_dir, '..', 'data', 'schedules.db')
    except NameError:
        # When running in notebook or interactive environment
        # Try different possible locations
        possible_paths = [
            'schedules.db',  # Current directory
            'data/schedules.db',  # data subdirectory
            '../data/schedules.db',  # parent/data
            '../../data/schedules.db',  # grandparent/data
            '/home/asj53/BOScheduling/schedules.db',  # Absolute path guess
            'myproject/data/schedules.db'  # Project structure
        ]
        
        for path in possible_paths:
            abs_path = os.path.abspath(path)
            if os.path.exists(abs_path):
                print(f"Found database at: {abs_path}")
                return abs_path
        
        # If none found, return the first option and let user know
        print("Database not found in common locations. Please update the path manually.")
        print("Tried these paths:")
        for path in possible_paths:
            print(f"  - {os.path.abspath(path)}")
        return os.path.abspath(possible_paths[0])

def verify_db_connection():
    """Verify database exists and is accessible."""
    if not os.path.exists(DB_PATH):
        print(f"❌ Database file does not exist at: {DB_PATH}")
        print(f"Current working directory: {os.getcwd()}")
        print("Please check the path or create the database first.")
        return False
    
    try:
        # Test connection
        test_engine = create_engine(DB_URL, echo=False)
        with test_engine.begin() as conn:
            conn.execute(text("SELECT 1"))
        print(f"✅ Database connection successful: {DB_PATH}")
        return True
    except Exception as e:
        print(f"❌ Database connection failed: {e}")
        return False

DB_PATH = get_db_path()
DB_URL = f"sqlite:///{DB_PATH}"
SEMESTER = "sp25"  # Update as needed

print(f"Database path: {DB_PATH}")
print(f"Database URL: {DB_URL}")
print(f"Database exists: {os.path.exists(DB_PATH)}")

# Verify connection on import
if not verify_db_connection():
    print("\n🔧 To fix this:")
    print("1. Update the database path in get_db_path() function")
    print("2. Or create the database at the expected location")
    print("3. Or run the database creation script first")
# ───────────────────────────────────────────────────────────────────────────────

engine = create_engine(DB_URL, echo=False)  # Set echo=True for debugging

def get_schedule_files(date_prefix: str, semester: str = SEMESTER) -> List[str]:
    """Get schedule IDs from database that match the given date prefix."""
    print('Getting schedules for prefix:', date_prefix)
    
    with engine.begin() as conn:
        # Fixed: Search in schedule_plots table instead of schedules table
        # Only include schedules that also have metrics (if you want to filter by semester)
        # OR just get all from schedule_plots if you don't need semester filtering
        
        if semester:
            # Option 1: Only get schedules that have metrics with the specified semester
            result = conn.execute(text("""
                SELECT DISTINCT sp.schedule_id 
                FROM schedule_plots sp
                JOIN metrics m ON sp.schedule_id = m.schedule_id
                WHERE sp.schedule_id LIKE :prefix 
                AND m.semester = :semester
                ORDER BY sp.schedule_id
            """), {
                "prefix": f"{date_prefix}_%",
                "semester": semester
            })
        else:
            # Option 2: Get all schedules from schedule_plots, regardless of metrics
            result = conn.execute(text("""
                SELECT DISTINCT schedule_id 
                FROM schedule_plots
                WHERE schedule_id LIKE :prefix 
                ORDER BY schedule_id
            """), {
                "prefix": f"{date_prefix}_%"
            })
        
        schedule_ids = [row.schedule_id for row in result]
        print(f"Found {len(schedule_ids)} schedules matching prefix '{date_prefix}_'")
        
        # Debug: show some examples if found
        if schedule_ids:
            print(f"Example schedule IDs found:")
            for i, sid in enumerate(schedule_ids[:5]):  # Show first 5
                print(f"  {i+1}. {sid}")
            if len(schedule_ids) > 5:
                print(f"  ... and {len(schedule_ids) - 5} more")
        
        return schedule_ids


def check_plot_exists(schedule_id: str, plot_type: str = 'sched_plot') -> bool:
    """Check if a plot exists in the database for the given schedule_id."""
    with engine.begin() as conn:
        result = conn.execute(text("""
            SELECT 1 FROM schedule_plots 
            WHERE schedule_id = :schedule_id 
            AND :plot_type IS NOT NULL
        """), {
            "schedule_id": schedule_id,
            "plot_type": plot_type
        })
        return result.fetchone() is not None

def update_plot_in_db(schedule_id: str, plot_type: str, plot_filename: str, semester: str = SEMESTER):
    """Update the plot information in the database."""
    with engine.begin() as conn:
        # Check if record exists
        existing = conn.execute(text("""
            SELECT schedule_id FROM schedule_plots 
            WHERE schedule_id = :schedule_id
        """), {"schedule_id": schedule_id}).fetchone()
        
        if existing:
            # Update existing record
            if plot_type == 'sched_plot':
                conn.execute(text("""
                    UPDATE schedule_plots 
                    SET sched_plot = :plot_filename
                    WHERE schedule_id = :schedule_id
                """), {
                    "plot_filename": plot_filename,
                    "schedule_id": schedule_id
                })
            elif plot_type == 'last_plot':
                conn.execute(text("""
                    UPDATE schedule_plots 
                    SET last_plot = 1
                    WHERE schedule_id = :schedule_id
                """), {"schedule_id": schedule_id})
        else:
            # Insert new record
            sched_plot = plot_filename if plot_type == 'sched_plot' else None
            last_plot = 1 if plot_type == 'last_plot' else 0
            
            conn.execute(text("""
                INSERT INTO schedule_plots (schedule_id, sched_plot, last_plot, semester)
                VALUES (:schedule_id, :sched_plot, :last_plot, :semester)
            """), {
                "schedule_id": schedule_id,
                "sched_plot": sched_plot,
                "last_plot": last_plot,
                "semester": semester
            })

def plot_exists_on_disk(schedule_id: str, plot_suffix: str = '', plots_dir: str = None) -> bool:
    """Check if plot file exists on disk."""
    if plots_dir is None:
        try:
            # When running as a script
            plots_dir = os.path.join(os.path.dirname(__file__), 'static', 'plots')
        except NameError:
            # When running in notebook - adjust this path as needed
            plots_dir = os.path.abspath('static/plots')  # Update this path
    
    plot_filename = f"{schedule_id}{plot_suffix}.png"
    plot_path = os.path.join(plots_dir, plot_filename)
    return os.path.exists(plot_path)

@lru_cache(maxsize=1)
def generate_plots_for_files(date_prefix: str, semester: str = SEMESTER):
    """Generate missing schedule and distribution plots for all files matching prefix."""
    schedule_ids = get_schedule_files(date_prefix, semester)
    print("PLOT SCHEDULE IDs:", schedule_ids)
    
    for schedule_id in schedule_ids:
        # Check and generate regular schedule plot
        if not plot_exists_on_disk(schedule_id):
            print(f"Generating schedule plot for {schedule_id}")
            get_plot(schedule_id, schedule_id)  # Assuming get_plot function exists
            # Update database after successful plot generation
            plot_filename = f"{schedule_id}.png"
            update_plot_in_db(schedule_id, 'sched_plot', plot_filename, semester)
        
        # Check and generate distribution plot
        if not plot_exists_on_disk(schedule_id, '_dist'):
            print(f"Generating distribution plot for {schedule_id}")
            last_day(schedule_id, schedule_id)  # Assuming last_day function exists
            # Update database after successful plot generation
            update_plot_in_db(schedule_id, 'last_plot', f"{schedule_id}_dist.png", semester)

def get_schedule_plot_info(schedule_id: str) -> dict:
    """Get plot information for a specific schedule from the database."""
    with engine.begin() as conn:
        result = conn.execute(text("""
            SELECT schedule_id, sched_plot, last_plot, semester
            FROM schedule_plots
            WHERE schedule_id = :schedule_id
        """), {"schedule_id": schedule_id})
        
        row = result.fetchone()
        if row:
            return {
                'schedule_id': row.schedule_id,
                'sched_plot': row.sched_plot,
                'has_last_plot': bool(row.last_plot),
                'semester': row.semester
            }
        return None

def get_all_plots_for_prefix(date_prefix: str, semester: str = SEMESTER) -> List[dict]:
    """Get all plot information for schedules matching the prefix."""
    with engine.begin() as conn:
        result = conn.execute(text("""
            SELECT sp.schedule_id, sp.sched_plot, sp.last_plot, sp.semester,
                   s.display_name, s.max_slot
            FROM schedule_plots sp
            JOIN schedules s ON sp.schedule_id = s.schedule_id
            WHERE sp.schedule_id LIKE :prefix
            AND sp.semester = :semester
            ORDER BY sp.schedule_id
        """), {
            "prefix": f"{date_prefix}_%",  # Changed from % to _%
            "semester": semester
        })
        
        plots = []
        for row in result:
            plots.append({
                'schedule_id': row.schedule_id,
                'sched_plot': row.sched_plot,
                'has_last_plot': bool(row.last_plot),
                'semester': row.semester,
                'display_name': row.display_name,
                'max_slot': row.max_slot
            })
        
        return plots

def cleanup_missing_plots(semester: str = SEMESTER):
    """Remove database entries for plots that no longer exist on disk."""
    try:
        # When running as a script
        plots_dir = os.path.join(os.path.dirname(__file__), 'static', 'plots')
    except NameError:
        # When running in notebook - adjust this path as needed
        plots_dir = os.path.abspath('static/plots')  # Update this path
    
    with engine.begin() as conn:
        # Get all plot records
        result = conn.execute(text("""
            SELECT schedule_id, sched_plot, last_plot
            FROM schedule_plots
            WHERE semester = :semester
        """), {"semester": semester})
        
        for row in result:
            schedule_id = row.schedule_id
            needs_update = False
            updates = {}
            
            # Check if sched_plot file exists
            if row.sched_plot:
                plot_path = os.path.join(plots_dir, row.sched_plot)
                if not os.path.exists(plot_path):
                    updates['sched_plot'] = None
                    needs_update = True
            
            # Check if distribution plot file exists
            if row.last_plot:
                dist_plot_path = os.path.join(plots_dir, f"{schedule_id}_dist.png")
                if not os.path.exists(dist_plot_path):
                    updates['last_plot'] = 0
                    needs_update = True
            
            # Update database if needed
            if needs_update:
                if 'sched_plot' in updates and 'last_plot' in updates:
                    conn.execute(text("""
                        UPDATE schedule_plots 
                        SET sched_plot = :sched_plot, last_plot = :last_plot
                        WHERE schedule_id = :schedule_id
                    """), {
                        "sched_plot": updates['sched_plot'],
                        "last_plot": updates['last_plot'],
                        "schedule_id": schedule_id
                    })
                elif 'sched_plot' in updates:
                    conn.execute(text("""
                        UPDATE schedule_plots 
                        SET sched_plot = :sched_plot
                        WHERE schedule_id = :schedule_id
                    """), {
                        "sched_plot": updates['sched_plot'],
                        "schedule_id": schedule_id
                    })
                elif 'last_plot' in updates:
                    conn.execute(text("""
                        UPDATE schedule_plots 
                        SET last_plot = :last_plot
                        WHERE schedule_id = :schedule_id
                    """), {
                        "last_plot": updates['last_plot'],
                        "schedule_id": schedule_id
                    })
                
                print(f"Updated plot records for {schedule_id}: {updates}")

# Example usage and testing function
def test_functions():
    """Test the helper functions with better error handling."""
    print("Testing helper functions...")
    
    # First verify database connection
    if not verify_db_connection():
        print("Cannot proceed with tests - database connection failed")
        return
    
    # Test getting schedule files
    test_prefix = "20250623"
    try:
        schedules = get_schedule_files(test_prefix)
        print(f"Found schedules: {schedules}")
        
        # Test getting plot info
        if schedules:
            plot_info = get_schedule_plot_info(schedules[0])
            print(f"Plot info for {schedules[0]}: {plot_info}")
        
        # Test getting all plots for prefix
        all_plots = get_all_plots_for_prefix(test_prefix)
        print(f"All plots for prefix: {len(all_plots)} found")
        
    except Exception as e:
        print(f"Error during testing: {e}")
        print("This might be due to missing tables or data in the database")

def create_database_tables():
    """Create the necessary tables if they don't exist."""
    print("Creating database tables...")
    
    try:
        with engine.begin() as conn:
            # Create schedules table
            conn.execute(text("""
                CREATE TABLE IF NOT EXISTS schedules (
                  schedule_id   TEXT    PRIMARY KEY,
                  display_name  TEXT,
                  max_slot      INTEGER
                );
            """))
            
            # Create metrics table  
            conn.execute(text("""
                CREATE TABLE IF NOT EXISTS metrics (
                  schedule_id             TEXT    PRIMARY KEY
                                             REFERENCES schedules(schedule_id),
                  conflicts               INTEGER,
                  quints                  INTEGER,
                  quads                   INTEGER,
                  four_in_five            INTEGER,
                  triple_in_24h           INTEGER,
                  triple_in_same_day      INTEGER,
                  three_in_four           INTEGER,
                  evening_morning_b2b     INTEGER,
                  other_b2b               INTEGER,
                  two_in_three            INTEGER,
                  singular_late           INTEGER,
                  two_large_gap           INTEGER,
                  avg_max                 FLOAT,
                  lateness                INTEGER,
                  size_cutoff             INTEGER,
                  reserved                INTEGER,
                  num_blocks              INTEGER,
                  alpha                   FLOAT,
                  gamma                   FLOAT,
                  delta                   FLOAT,
                  vega                    FLOAT,
                  theta                   FLOAT,
                  large_block_size        FLOAT,
                  large_exam_weight       FLOAT,
                  large_block_weight      FLOAT,
                  large_size_1            FLOAT,
                  large_cutoff_freedom    FLOAT,
                  tradeoff                FLOAT,
                  flpens                  FLOAT,
                  semester                TEXT
                );
            """))
            
            # Create schedule_plots table (renamed from schedule_details)
            conn.execute(text("""
                CREATE TABLE IF NOT EXISTS schedule_plots (
                  schedule_id   TEXT    NOT NULL
                                           REFERENCES schedules(schedule_id),
                  sched_plot    TEXT,
                  last_plot     INTEGER,
                  semester      TEXT,
                  PRIMARY KEY (schedule_id)
                );
            """))
            
        print("✅ Database tables created successfully")
        
    except Exception as e:
        print(f"❌ Error creating tables: {e}")

# Manual path override function
def set_database_path(path: str):
    """Manually set the database path if auto-detection fails."""
    global DB_PATH, DB_URL, engine
    
    DB_PATH = os.path.abspath(path)
    DB_URL = f"sqlite:///{DB_PATH}"
    engine = create_engine(DB_URL, echo=False)
    
    print(f"Database path updated to: {DB_PATH}")
    print(f"Database exists: {os.path.exists(DB_PATH)}")
    
    return verify_db_connection()

if __name__ == "__main__":
    test_functions()

Found database at: /home/asj53/BOScheduling/myproject/data/schedules.db
Database path: /home/asj53/BOScheduling/myproject/data/schedules.db
Database URL: sqlite:////home/asj53/BOScheduling/myproject/data/schedules.db
Database exists: True
✅ Database connection successful: /home/asj53/BOScheduling/myproject/data/schedules.db
Testing helper functions...
✅ Database connection successful: /home/asj53/BOScheduling/myproject/data/schedules.db
Getting schedules for prefix: 20250623
Found 0 schedules matching prefix '20250623_'
Found schedules: []
All plots for prefix: 0 found


In [16]:
import sqlite3
from sqlalchemy import create_engine, text

# Use the same database path from your existing code
DB_PATH = get_db_path()
DB_URL = f"sqlite:///{DB_PATH}"
engine = create_engine(DB_URL, echo=True)  # Enable echo to see SQL queries

def debug_specific_schedule_id():
    """Debug the specific schedule ID you mentioned."""
    target_schedule_id = "20250614_065847i2-5d4672b13683d7963d5f2c95d1a5aded"
    target_semester = "sp25"
    
    print(f"Looking for schedule ID: {target_schedule_id}")
    print(f"Target semester: {target_semester}")
    print("="*80)
    
    with engine.begin() as conn:
        # 1. Check if the schedule exists in schedules table
        print("1. Checking schedules table...")
        result = conn.execute(text("""
            SELECT schedule_id, display_name, max_slot 
            FROM schedules 
            WHERE schedule_id = :schedule_id
        """), {"schedule_id": target_schedule_id})
        
        schedule_row = result.fetchone()
        if schedule_row:
            print(f"✅ Found in schedules table:")
            print(f"   ID: {schedule_row.schedule_id}")
            print(f"   Display Name: {schedule_row.display_name}")
            print(f"   Max Slot: {schedule_row.max_slot}")
        else:
            print("❌ NOT found in schedules table")
        
        print()
        
        # 2. Check if it exists in metrics table
        print("2. Checking metrics table...")
        result = conn.execute(text("""
            SELECT schedule_id, semester, conflicts, lateness 
            FROM metrics 
            WHERE schedule_id = :schedule_id
        """), {"schedule_id": target_schedule_id})
        
        metrics_row = result.fetchone()
        if metrics_row:
            print(f"✅ Found in metrics table:")
            print(f"   ID: {metrics_row.schedule_id}")
            print(f"   Semester: {metrics_row.semester}")
            print(f"   Conflicts: {metrics_row.conflicts}")
            print(f"   Lateness: {metrics_row.lateness}")
        else:
            print("❌ NOT found in metrics table")
        
        print()
        
        # 3. Check if semester matches
        if metrics_row and metrics_row.semester != target_semester:
            print(f"⚠️  Semester mismatch! Expected '{target_semester}', found '{metrics_row.semester}'")
        
        # 4. Try the JOIN query that was failing
        print("3. Testing the original JOIN query...")
        result = conn.execute(text("""
            SELECT DISTINCT s.schedule_id, m.semester
            FROM schedules s
            JOIN metrics m ON s.schedule_id = m.schedule_id
            WHERE s.schedule_id = :schedule_id 
            AND m.semester = :semester
        """), {
            "schedule_id": target_schedule_id,
            "semester": target_semester
        })
        
        join_row = result.fetchone()
        if join_row:
            print(f"✅ JOIN query successful:")
            print(f"   ID: {join_row.schedule_id}")
            print(f"   Semester: {join_row.semester}")
        else:
            print("❌ JOIN query returned no results")
        
        print()

def debug_prefix_search():
    """Debug the prefix search that's been failing."""
    date_prefix = "20250614"
    target_semester = "sp25"
    
    print(f"Testing prefix search for: {date_prefix}")
    print(f"Target semester: {target_semester}")
    print("="*80)
    
    with engine.begin() as conn:
        # 1. Check schedules that start with the date prefix
        print("1. All schedule IDs starting with prefix (no JOIN)...")
        result = conn.execute(text("""
            SELECT schedule_id 
            FROM schedule_plots 
            WHERE schedule_id LIKE :prefix
            ORDER BY schedule_id
            LIMIT 10
        """), {"prefix": f"{date_prefix}_%"})
        
        schedules_only = [row.schedule_id for row in result]
        print(f"Found {len(schedules_only)} schedules starting with '{date_prefix}_':")
        for sid in schedules_only:
            print(f"   {sid}")
        
        print()
        
        # 2. Check what semesters exist for these schedules
        if schedules_only:
            print("2. Checking semesters for these schedules...")
            placeholders = ','.join([':param' + str(i) for i in range(len(schedules_only))])
            params = {f'param{i}': sid for i, sid in enumerate(schedules_only)}
            
            result = conn.execute(text(f"""
                SELECT schedule_id, semester 
                FROM metrics 
                WHERE schedule_id IN ({placeholders})
            """), params)
            
            semester_info = {row.schedule_id: row.semester for row in result}
            
            for sid in schedules_only:
                if sid in semester_info:
                    semester = semester_info[sid]
                    match_indicator = "✅" if semester == target_semester else "❌"
                    print(f"   {match_indicator} {sid} -> semester: {semester}")
                else:
                    print(f"   ❌ {sid} -> NO METRICS FOUND")
        
        print()
        
        # 3. Try the full JOIN with prefix
        print("3. Testing full JOIN query with prefix...")
        result = conn.execute(text("""
            SELECT DISTINCT s.schedule_id, m.semester
            FROM schedule_plots s
            JOIN metrics m ON s.schedule_id = m.schedule_id
            WHERE s.schedule_id LIKE :prefix 
            AND m.semester = :semester
            ORDER BY s.schedule_id
        """), {
            "prefix": f"{date_prefix}_%",
            "semester": target_semester
        })
        
        join_results = [row.schedule_id for row in result]
        print(f"JOIN query returned {len(join_results)} results:")
        for sid in join_results:
            print(f"   {sid}")

def show_all_semesters():
    """Show all unique semesters in the database."""
    print("All semesters in metrics table:")
    print("="*40)
    
    with engine.begin() as conn:
        result = conn.execute(text("""
            SELECT semester, COUNT(*) as count 
            FROM metrics 
            GROUP BY semester 
            ORDER BY semester
        """))
        
        for row in result:
            print(f"   {row.semester}: {row.count} schedules")

def show_sample_schedule_ids():
    """Show sample schedule IDs to understand the format."""
    print("Sample schedule IDs from database:")
    print("="*50)
    
    with engine.begin() as conn:
        result = conn.execute(text("""
            SELECT schedule_id 
            FROM schedule_plots 
            ORDER BY schedule_id 
            LIMIT 10
        """))
        
        for i, row in enumerate(result, 1):
            print(f"   {i}. {row.schedule_id}")

# Main debug function
def full_debug():
    """Run all debug functions."""
    print("FULL DATABASE DEBUG")
    print("="*80)
    
    try:
        show_sample_schedule_ids()
        print("\n")
        
        show_all_semesters()
        print("\n")
        
        debug_specific_schedule_id()
        print("\n")
        
        debug_prefix_search()
        
    except Exception as e:
        print(f"Error during debug: {e}")
        print("Make sure to update DB_PATH at the top of this script!")

if __name__ == "__main__":
    full_debug()

Found database at: /home/asj53/BOScheduling/myproject/data/schedules.db
FULL DATABASE DEBUG
Sample schedule IDs from database:
2025-06-28 07:25:52,018 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-06-28 07:25:52,019 INFO sqlalchemy.engine.Engine 
            SELECT schedule_id 
            FROM schedule_plots 
            ORDER BY schedule_id 
            LIMIT 10
        
2025-06-28 07:25:52,019 INFO sqlalchemy.engine.Engine [generated in 0.00076s] ()
   1. 20250614_065847i1-30830ed058547eee431b8eed83a4feda
   2. 20250614_065847i2-5d4672b13683d7963d5f2c95d1a5aded
   3. 20250614_065847i3-7eee9a7019805f42ad398408c5bd6b1c
   4. 20250614_065847i4-6286463d8ea88823a71ad71e35087207
   5. 20250614_065847i5-5414981f3563c30b9f4ab1710ba92f7e
   6. 20250614_071641i1-e1156f71788526c21089f219d9a4a8c8
   7. 20250614_071641i1-e1156f71788526c21089f219d9a4a8c8_dist
   8. 20250614_071641i2-efb1c55a835b028c53c39c78d16720ca
   9. 20250614_071641i2-efb1c55a835b028c53c39c78d16720ca_dist
   10. 2025061